In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import regex
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import re
import string

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import MaxPooling1D
from keras.layers import Conv1D


from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier



from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score



In [28]:
# %run SheidaTalei_Preprocessing_FinalProject.ipynb

In [29]:
def removeEmptyRows(fileName):
    df = pd.read_csv(fileName, encoding='utf-8-sig')
    df = df.dropna(subset=['text'], how='all', axis=0) 
    df.to_csv(fileName, header=True, encoding='utf-8-sig',  index=False)
    
    return df

In [ ]:
def convertSequenceWordsToText(input_list, dataFrame, fileName):
    X_and_Y = pd.read_csv(dataFrame , encoding="utf-8")
    text = ''
    counter = 0
    for item in input_list:
        for i in range(len(item)):
            text += list(item)[i] + ' '
        print(counter)    
        X_and_Y.loc [counter, 'text'] = text.strip()
        X_and_Y.to_csv(fileName ,index=False, encoding='utf-8-sig')
        counter = counter + 1
        text =''

In [ ]:
convertSequenceWordsToText(lemmatize_tweet_list, '/content/drive/My Drive/Final/X_and_Y.csv' , '/content/drive/My Drive/Final/X_and_Y_train_LSTM.csv')

In [ ]:
convertSequenceWordsToText(lemmatize_tweet_list_test,'/content/drive/My Drive/Final/X_and_Y_test.csv', '/content/drive/My Drive/Final/X_and_Y_test_LSTM.csv')

In [30]:

removeEmptyRows('/content/drive/My Drive/Final/X_and_Y_train_LSTM.csv')

X_and_Y = pd.read_csv('/content/drive/My Drive/Final/X_and_Y_train_LSTM.csv',encoding="utf-8")
X_train = X_and_Y.text
Y_train= X_and_Y.Label

# #Tokenization
# train_data_tokenize_list = tokenization (X)
#------------------------------------------------------test---------------------------------------------------------------------
#step 2: Tokenizing data


removeEmptyRows('/content/drive/My Drive/Final/X_and_Y_test_LSTM.csv')
X_and_Y_test = pd.read_csv('/content/drive/My Drive/Final/X_and_Y_test_LSTM.csv',encoding="utf-8")
X_test = X_and_Y_test.text
Y_test= X_and_Y_test.Label

# #Tokenization
# train_data_tokenize_list_test = tokenization (X_test)

In [31]:
#converting words to vectors
max_feature = 300
tokenizer = Tokenizer(num_words = max_feature, lower = True, split = ' ')
tokenizer.fit_on_texts(X_train )


X = tokenizer.texts_to_sequences(X_train)

#pad sequence is used to ensure that all sequences in a list have the same length.
X = pad_sequences(sequences = X, maxlen = max_feature, padding = 'pre')

In [32]:
def createLstmModel(units = 200, dropout = 0.5 ):
    
    
   #https://www.tensorflow.org/tutorials/text/text_classification_rnn
        
    #Adding Layers
    model = Sequential()
    model.add(Embedding(input_dim=300, output_dim=units))

    model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))  
   
    # model.add(LSTM(units = units , dropout = 0.2, return_sequences=True ))
    # model.add(Dense(units = units, activation="relu"))
    # model.add(Dropout(rate = dropout))


  
    model.add(LSTM(units = units , dropout = 0.2))
    model.add(Dense(units = units, activation="relu"))
    model.add(Dropout(rate = dropout))

    model.add(Dense(units = 1 ,activation="sigmoid"))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(), 'accuracy'])
#     model.fit(X, Y_train)
#     model.summary()
    return model


In [24]:
# Fit LSTM model
#source: https://apmonitor.com/do/index.php/Main/LSTMNetwork
# model = createLstmModel()
# history = model.fit(X, Y_train , epochs = 1)

# plt.figure()
# plt.ylabel('loss'); plt.xlabel('epoch')
# plt.semilogy(history.history['loss'])

In [33]:
model = KerasClassifier(build_fn=createLstmModel, epochs=1)


pipeline =  [
#         ('tokenizer', Tokenizer(num_words = max_feature, lower = True, split = ' ')),
#         ('pad', pad_sequences(sequences = X , maxlen = max_feature, padding = 'pre')),
        ('estimator', model)
    ]

parameters = {
#     'tokenizer__num_words': (100, 200, 300),
#     'pad__maxlen':( 200, 300, 400),
    
    'estimator__dropout': (0.5,0.1, 0.01),
    'estimator__units' :(100,  300, 700, 1000)
}

grid_search_tune = GridSearchCV(Pipeline(pipeline), parameters, cv=3, verbose=1)
print('*')
LSTM = grid_search_tune.fit(X, Y_train)



*
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


104/104 [==============================] - 3s 28ms/step - loss: 0.3594 - auc_91: 0.7019 - accuracy: 0.8751


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  7.3min finished


312/312 [==============================] - 14s 45ms/step - loss: 0.4150 - auc_92: 0.7432 - accuracy: 0.8387


In [18]:
print (grid_search_tune.best_estimator_.steps)

print ('Best Parameter for LSTM ',grid_search_tune.best_params_)
print('Best Score for LSTM: ',grid_search_tune.best_score_)

[('estimator', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f0eaa176630>)]
Best Parameter for LSTM  {'estimator__dropout': 0.5, 'estimator__units': 100}
Best Score for LSTM:  0.8378486831982931


In [19]:
max_feature = 200
tokenizer = Tokenizer(num_words = max_feature, lower = True, split = ' ')
tokenizer.fit_on_texts(X_test )


X = tokenizer.texts_to_sequences(X_test)

#pad sequence is used to ensure that all sequences in a list have the same length.
X = pad_sequences(sequences = X, maxlen = max_feature, padding = 'pre')

In [20]:
y_predict =grid_search_tune.predict(X)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [21]:
print('Accuracy: ' , accuracy_score( Y_test,y_predict))
#Source: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
print('F1-score: ', f1_score(Y_test, y_predict))
#source: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
print('roc_auc_score: ', roc_auc_score(Y_test, y_predict))

Accuracy:  0.763250528009655
F1-score:  0.02079866888519135
roc_auc_score:  0.49631305949821886
